In [12]:
import requests
import pandas as pd
from tqdm.auto import tqdm

In [27]:
def get_spin(text):
    try:
        response = requests.post('https://spinbot-back.azurewebsites.net/spin/rewrite-text', json={'text': text, 'x_spin_cap_words': False})
        resp = response.json()
    except:
        return ''
    return resp

In [28]:
df = pd.read_csv(('./real_passages_100k.csv'))
sample = df.sample(n=1000, random_state=1)

In [29]:
paraphrases = []
for i, item in tqdm(sample.iterrows(), total=len(sample)):
    paraphrases.append(
        get_spin(item['passages'])
    )

  0%|          | 0/1000 [00:00<?, ?it/s]

In [30]:
from difflib import SequenceMatcher

SIMILARITY_THRESHOLD = 0.8

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [31]:
clean_paraphrases = []
for para, original in zip(paraphrases, list(sample['passages'])):
    if para.strip() == '':
        continue
    if similar(para, original) <= SIMILARITY_THRESHOLD:
        clean_paraphrases.append(para)

In [32]:
len(clean_paraphrases)

901

In [34]:
pd.DataFrame({
    'passages': paraphrases
}).to_csv('./spinbot_paraphrases.csv', index=False)